In [1]:
import sys
sys.path.append('../')
import pnn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
import onnxruntime as ort
from io import BytesIO

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
import onnx
import onnxsim


In [4]:
# 只修改了哈达玛乘的部分
def gen_pattern_replace_and_matcher_for_loop_pnn_mul(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return reducey_x[redundancy_x_row] * reducey_x[reducey_x_col],\
              non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],\
              reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col]


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      cross_term = torch.sum(cross_term, dim = 2)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [5]:
# 联合修改了哈达玛乘和reduce部分
def gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim = -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = 2)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [6]:
# 只修改了reduce部分
def gen_pattern_replace_and_matcher_for_loop_pnn_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim = -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = 2)

    def pn(self,x):
      cross_term =  x[getitem_node_args[0]] * x[getitem_node_args[1]]
      redundancy_part_pn = cross_term[0,:self.redundancy_cross_part_total]
      non_redundancy_part_pn = cross_term[:,self.redundancy_cross_part_total:self.redundancy_cross_part_total + self.non_redundancy_cross_part_total]
      mixed_part_pn = cross_term[:,self.redundancy_cross_part_total + self.non_redundancy_cross_part_total:]
      return torch.sum(redundancy_part_pn.unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_part_pn,dim = -1),\
              torch.sum(mixed_part_pn,dim = 2)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(x)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [7]:
# 全模型改写
def gen_pattern_replace_and_matcher_for_loop_pnn(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64)),False)
      self.unredency_offset = nn.Parameter(torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64)),False)
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]

      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

    def pn(self,reducey_x,unredency_x):
       redency_x_row, reducey_x_col = list(),list()
       unredency_x_row, unredency_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redency_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             unredency_x_row.append(i),unredency_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redency_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = 2),\
              torch.sum(unredency_x[:,unredency_x_row] * unredency_x[:,unredency_x_col],dim = 2),\
              torch.sum(reducey_x[mixed_x_row] * unredency_x[:,mixed_x_col],dim = 2)


    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_part_embed = self.embed(redency_part)
      unredency_part_embed = self.embed(unredency_part)
      redency_part_pn, unredency_part_pn, mixed_part_pn = self.pn(redency_part_embed,unredency_part_embed)
      
      
      return self.redency_linear(self.embed(redency_part).view(-1,self.embed_dim * self.num_prefix)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.embed_dim * self.num_sufix)) +\
         self.redency_cross_part_linear(redency_part_pn) + self.mixed_cross_part_linear(mixed_part_pn) + self.unredency_cross_part_linear(unredency_part_pn)
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [8]:
# 只改写linear
def gen_pattern_replace_and_matcher_for_loop_pnn_linear(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.num_fields * self.embed_dim
      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

    def pn(self,x):
         cross_term =  x[getitem_node_args[0]] * x[getitem_node_args[1]]
         return torch.sum(cross_term, dim = 2)

    def forward(self,x):
      x = self.embed(x + self.offsets)
      cross_term = self.pn(x)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      len_redundancy = self.embed_dim * self.num_prefix
      len_nonredundancy = len_redundancy + self.embed_dim * self.num_sufix
      len_pn_redundancy = len_nonredundancy + self.redency_cross_part_total
      len_pn_nonredundancy = len_pn_redundancy + self.unredency_cross_part_total
      
      
      
      
      
      return self.redency_linear(x[0,:len_redundancy]) + self.unredency_linear(x[:,len_redundancy : len_nonredundancy]) +\
         self.redency_cross_part_linear(x[0,len_nonredundancy:len_pn_redundancy]) +\
         self.mixed_cross_part_linear(x[:,len_pn_nonredundancy:]) + self.unredency_cross_part_linear(x[:,len_pn_redundancy:len_pn_nonredundancy])
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [9]:
# 测试对mul算子和reduce算子做单独改写的效果
def gen_pattern_replace_and_matcher_for_loop_pnn_single_mul_reduce(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn1(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return (reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),\
              non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],\
              reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col]

    def pn(self,x):
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn1(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      redundancy_part_pn = cross_term[0,:self.redundancy_cross_part_total]
      non_redundancy_part_pn = cross_term[:,self.redundancy_cross_part_total:self.redundancy_cross_part_total + self.non_redundancy_cross_part_total]
      mixed_part_pn = cross_term[:,self.redundancy_cross_part_total + self.non_redundancy_cross_part_total:]
      return torch.sum(redundancy_part_pn.unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_part_pn,dim = -1),\
              torch.sum(mixed_part_pn,dim = -1)


    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(x)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 1)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

In [10]:
# 使用优化的方法改写
def gen_pattern_replace_and_matcher_for_optimize(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  # getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1] * num_field
          self.interaction_units = int(num_field * (num_field - 1) / 2)
          self.triu_mask = nn.Parameter(torch.triu(torch.ones(num_field, num_field), 1).bool(),
                                    requires_grad=False) 
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,feature_emb):
        inner_product_matrix = torch.bmm(feature_emb, feature_emb.transpose(1, 2))
        triu_values = torch.masked_select(inner_product_matrix, self.triu_mask)
        return triu_values.view(-1, self.interaction_units)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return x
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets = offsets_val
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.embed_output_dim = self.embed_dim * self.num_fields
      self.redundancy_mask = nn.Parameter(torch.triu(torch.ones(self.num_prefix, self.num_prefix), 1).bool(),
                                    requires_grad=False)
      self.non_redundancy_mask = nn.Parameter(torch.triu(torch.ones(self.num_sufix, self.num_sufix), 1).bool(),
                                    requires_grad=False)
      # self.ll = nn.Linear(linear_node_weight.shape[1],linear_node_weight.shape[0],bias = True)

    def pn(self,redundancy_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix

       redundancy_interaction_units = int(prefix * (prefix - 1) / 2)
       non_redundancy_interaction_units = int(sufix * (sufix - 1) / 2)
       mixed_interaction_units = int(prefix * sufix)
       
       non_redundancy_x_t = non_redundancy_x.transpose(1, 2)
       
       redundancy_inner_product_matrix = redundancy_x @ redundancy_x.transpose(-2,-1)
       non_redundancy_inner_product_matrix = torch.bmm(non_redundancy_x, non_redundancy_x_t)
       
       mixed_redundancy_inner_product_matrix = redundancy_x @ non_redundancy_x_t
       
       redundancy_triu_values = torch.masked_select(redundancy_inner_product_matrix, self.redundancy_mask)
       non_redundancy_triu_values = torch.masked_select(non_redundancy_inner_product_matrix,self.non_redundancy_mask)
       
       return redundancy_triu_values.view(-1,redundancy_interaction_units),\
              non_redundancy_triu_values.view(-1,non_redundancy_interaction_units),\
              mixed_redundancy_inner_product_matrix.view(-1,mixed_interaction_units)
    def forward(self,x):
      x = self.embed(x + self.offsets)
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      # cross_term = torch.sum(cross_term, dim = 2)
      x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
      return x
    
  
  return pattern,ReplacementClass(),_match

# 负载生成

In [11]:
# 只修改哈达玛积
def workload_pnn_mul(dim,redundancy,mlp = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [12]:
# 联合修改哈达玛积和reduce
def workload_pnn_mul_reduce(dim,redundancy,mlp = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [13]:
# 只修改reduce
def workload_pnn_reduce(dim,redundancy,mlp = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [14]:
# 修改全部模型
def workload_pnn(dim,redundancy,mlp = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlp,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [15]:
# 只修改linear
def workload_pnn_linear(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_linear(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [16]:
# 测试对mul算子和reduce算子做单独改写的效果

def workload_pnn_single_mul_reduce(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_single_mul_reduce(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [17]:
# 测试对mul算子和reduce算子做单独改写的效果

def workload_pnn_optimize(dim,redundancy,mlps = [400,400,400]):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1,"innerV2")

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(100)],dim,mlps,0.1,"innerV2")
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_optimize(pnn_model_traced_modify,
                                                                      (0,slice(None,redundancy,None)),(slice(None,None,None),slice(redundancy,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

# 测试

In [18]:
ori , modify = workload_pnn_mul(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [19]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.18 s ± 78.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [20]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.81 s ± 27.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [23]:
ori , modify = workload_pnn_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [24]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.19 s ± 46.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [25]:
ori , modify = workload_pnn_linear(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [26]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


3.18 s ± 39.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [27]:
ori , modify = workload_pnn_mul_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [28]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


1.1 s ± 32.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [31]:
ori , modify = workload_pnn_mul_reduce(128,40,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [32]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.34 s ± 47.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [33]:
ori , modify = workload_pnn_mul_reduce(128,20,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [34]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.79 s ± 18.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [29]:
ori , modify = workload_pnn(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [30]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


994 ms ± 22.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [35]:
ori , modify = workload_pnn(128,40,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [36]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.3 s ± 32.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [37]:
ori , modify = workload_pnn(128,20,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [38]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.76 s ± 28.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


全部改写和只改写关键算子以后性能差距不大

In [24]:
ori.to(device)
modify.to(device)

ProductNeuralNetworkModel(
  (mlp): Module(
    (mlp): Module(
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=1000, out_features=400, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=400, out_features=400, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.1, inplace=False)
      (9): Linear(in_features=400, out_features=1, bias=True)
    )
  )
  (embed): Embedding(10000, 128)
  (redency_linear): Linear(in_features=10240, out_features=1000, bias=False)
  (unredency_linear): Linear(in_features=2560, out_features=1000, bias=False)
  (redency_cross_part_linear): Linear(in_features=3160, out_features=1000, bias=True)
  (mixed_cross_part_linear): Linear(in_features=1600, out_features=1000, bias=False)
  (unredency_cross_part_linear): Linear(in_features=190, out_features=1000, bias=False)
)

In [27]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

The slowest run took 5.82 times longer than the fastest. This could mean that an intermediate result is being cached.
102 ms ± 15.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [28]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

The slowest run took 4.29 times longer than the fastest. This could mean that an intermediate result is being cached.
27.2 ms ± 3.84 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [29]:
ori , modify = workload_pnn(128,30,[1000,400,400])

now gen workload of pnn with config: dim: 128


In [30]:
ori.to(device)
modify.to(device)

ProductNeuralNetworkModel(
  (mlp): Module(
    (mlp): Module(
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=1000, out_features=400, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=400, out_features=400, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.1, inplace=False)
      (9): Linear(in_features=400, out_features=1, bias=True)
    )
  )
  (embed): Embedding(10000, 128)
  (redency_linear): Linear(in_features=3840, out_features=1000, bias=False)
  (unredency_linear): Linear(in_features=8960, out_features=1000, bias=False)
  (redency_cross_part_linear): Linear(in_features=435, out_features=1000, bias=True)
  (mixed_cross_part_linear): Linear(in_features=2100, out_features=1000, bias=False)
  (unredency_cross_part_linear): Linear(in_features=2415, out_features=1000, bias=False)
)

In [31]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

The slowest run took 5.49 times longer than the fastest. This could mean that an intermediate result is being cached.
102 ms ± 15.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [32]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

80.6 ms ± 5.71 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [33]:
ori , modify = workload_pnn(32,30,[1000,400,400])

now gen workload of pnn with config: dim: 32


In [35]:
ori.to(device)
modify.to(device)

ProductNeuralNetworkModel(
  (mlp): Module(
    (mlp): Module(
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=1000, out_features=400, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=400, out_features=400, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.1, inplace=False)
      (9): Linear(in_features=400, out_features=1, bias=True)
    )
  )
  (embed): Embedding(10000, 32)
  (redency_linear): Linear(in_features=960, out_features=1000, bias=False)
  (unredency_linear): Linear(in_features=2240, out_features=1000, bias=False)
  (redency_cross_part_linear): Linear(in_features=435, out_features=1000, bias=True)
  (mixed_cross_part_linear): Linear(in_features=2100, out_features=1000, bias=False)
  (unredency_cross_part_linear): Linear(in_features=2415, out_features=1000, bias=False)
)

In [36]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():ori(x)

The slowest run took 5.44 times longer than the fastest. This could mean that an intermediate result is being cached.
31.3 ms ± 4.7 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [37]:
x = torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long).to(device)
%timeit -n 1 -r 30 with torch.no_grad():modify(x)

26 ms ± 1.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [30]:
ori , modify = workload_pnn_single_mul_reduce(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [32]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


2.59 s ± 21.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [41]:
ori , modify = workload_pnn_optimize(128,80,[1000,400,400])

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [67]:
torch.onnx.export(
    ori,  # 模型实例
    torch.randint(low=0, high=88, size=(4096,100)),  # 示例输入（假设是图像数据）
    'pnn_opt_ori.onnx',  )

In [68]:
onnx_model = onnx.load('pnn_opt_ori.onnx')
model_simplified, check = onnxsim.simplify(onnx_model)
onnx.save(model_simplified, 'pnn_opt_ori_smi.onnx')


In [69]:
session = ort.InferenceSession('pnn_opt_ori_smi.onnx')


In [70]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Add_0' : np.random.randint(low=0, high=88, size=(4096, 100), dtype=np.int64)})

198 ms ± 25.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [47]:

session = ort.InferenceSession('pnn_opt_ori')


In [50]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Add_0' : np.random.randint(low=0, high=88, size=(4096, 100), dtype=np.int64)})

196 ms ± 21.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [38]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


418 ms ± 8.76 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [71]:
torch.onnx.export(
    modify,  # 模型实例
    torch.randint(low=0, high=88, size=(4096,100)),  # 示例输入（假设是图像数据）
    'pnn_opt_modify.onnx',  )

In [72]:
onnx_model = onnx.load('pnn_opt_modify.onnx')
model_simplified, check = onnxsim.simplify(onnx_model)


In [73]:
onnx.save(model_simplified, 'pnn_opt_modify_sim.onnx')


In [74]:
session = ort.InferenceSession('pnn_opt_modify_sim.onnx')


In [75]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Add_0' : np.random.randint(low=0, high=88, size=(4096, 100), dtype=np.int64)})

115 ms ± 14.7 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [39]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))


154 ms ± 9.06 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [58]:
x = np.random.randint(low=0, high=88, size=(4096, 100), dtype=np.int64)

In [59]:
%timeit -n 1 -r 30 session.run(None,{'onnx::Add_0' : x})

186 ms ± 2.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
